In [5]:
#Loading all the needed Packages
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [6]:
#Load csv files 
HourlyMCP = pd.read_csv('../Data/ModelData/HourlyMCPDemo.csv', delimiter=',')
Fuel_Cost_Forecast_Normalized = pd.read_csv('../Data/ModelData/FuelCost_PriceDevelopment50years.csv', delimiter=',')
Generation_Asset_Data_New = pd.read_csv('../Data/ModelData/Generators_AssetData_New.csv', delimiter=',')
VRE_Generation_profile = pd.read_csv('../Data/ModelData/VRE_YearlyGenerationProfile_Normalized.csv', delimiter=',')

#Define budget 
budget = 1000000

In [137]:
#show the first 5 rows of the data
#HourlyMCP.head()
#Fuel_Cost_Forecast_Normalized.head()
Generation_Asset_Data_New.head()
#VRE_Generation_profile.head()

#print entire data
#print(HourlyMCP)
#print(Fuel_Cost_Forecast_Normalized)
print(Generation_Asset_Data_New)
#print(VRE_Generation_profile)

    Unit #  Zone Technology  P_max (MW)  P_min (MW)  P_max current (MW)  \
0        1     0       Coal         152       30.40                   0   
1        2     0       Coal         152       30.40                   0   
2        3     0       Coal         200       75.00                   0   
3        4     0        Gas         200      206.85                   0   
4        5     0        Gas          60       12.00                   0   
5        6     0       Coal         155       54.25                   0   
6        7     0       Coal         155       54.25                   0   
7        8     1       Wind         200      100.00                   0   
8        9     1       Wind         200      100.00                   0   
9       10     1         PV         200      300.00                   0   
10      11     1       Wind         200      108.50                   0   
11      12     1       Wind         200      140.00                   0   
12      13     0       Wi

In [7]:
#Define modelling time horizon
#Amount of years based on HourlyMCP data
Y = HourlyMCP['Year'].nunique()

#Amount of seasons in a year based on HourlyMCP data
S = HourlyMCP['Season'].nunique()

#Amount of hours in a season
H = HourlyMCP[HourlyMCP['Season'] == 1].shape[0] // Y

#Print the amount of years, seasons and hours
# print("Amount of years: ", Y)
# print("Amount of seasons: ", S)
# print("Amount of hours in a season: ", H)

In [8]:
#Initialize the model
m = gp.Model("InvestmentModel")

#Define the decision variables
#Capacity Investment in invetment opportunity i defined by Generation_Asset_Data_New['Unit #']
Inv_NewCap = m.addVars(Generation_Asset_Data_New['Unit #'], vtype=GRB.CONTINUOUS, name="Inv_NewCap")
#Energy generated by investment opportunity i in hour h of season s of year y
P_NewCap = m.addVars(Generation_Asset_Data_New['Unit #'], Y, S, H, vtype=GRB.CONTINUOUS, name="P_NewCap")

#Define the objective function
# Objective function is to maximize the profit for operation in y years and s seasons and h hours and subtract investment cost
m.setObjective(73*
    gp.quicksum(
        gp.quicksum(
            gp.quicksum(
                (P_NewCap[i, y, s, h] * HourlyMCP.loc[(HourlyMCP['Year'] == y+1) & (HourlyMCP['Season'] == s+1) & (HourlyMCP['Hour'] == h+1), 'MCP'].values[0] - Generation_Asset_Data_New.loc[Generation_Asset_Data_New['Unit #'] == i, 'C_i ($/MWh)'].values[0])
                for h in range(H)
            )
            for s in range(S)
        )
        for y in range(Y)
        for i in Generation_Asset_Data_New['Unit #']
    ) - gp.quicksum(Inv_NewCap[i] * Generation_Asset_Data_New.loc[Generation_Asset_Data_New['Unit #'] == i, 'C_CapInv ($/MW)'].values[0] for i in Generation_Asset_Data_New['Unit #']),
    GRB.MAXIMIZE
)

#Define the constraints
#Investment constraint
m.addConstrs((Inv_NewCap[i] >= 0 for i in Generation_Asset_Data_New['Unit #']), "InvestmentMin")
m.addConstrs((Inv_NewCap[i] <= Generation_Asset_Data_New.loc[Generation_Asset_Data_New['Unit #'] == i, 'MaxInv (MW)'].values[0] for i in Generation_Asset_Data_New['Unit #']), "InvestmentMax")

#Budget constraint
m.addConstr(gp.quicksum(Inv_NewCap[i] * Generation_Asset_Data_New.loc[Generation_Asset_Data_New['Unit #'] == i, 'C_CapInv ($/MW)'].values[0] for i in Generation_Asset_Data_New['Unit #']) <= budget, "Budget")

#Generation constraint for y years, s seasons and h hours
m.addConstrs(
    (
        P_NewCap[i, y, s, h] <= Inv_NewCap[i]
        for i in Generation_Asset_Data_New['Unit #']
        for y in range(Y)
        for s in range(S)
        for h in range(H)
    ),
    "GenMax"
)

#VRE constraint for y years, s seasons and h hours
#generation constraint for assets with technology=PV 
m.addConstrs(
    (
        P_NewCap[i, y, s, h] <= Inv_NewCap[i] * VRE_Generation_profile.loc[
            (VRE_Generation_profile['Hour'] == h+1) & (VRE_Generation_profile['Season'] == s+1), 'PVZ1'
        ].values[0]
        for y in range(Y)
        for s in range(S)
        for h in range(H)
        for i in Generation_Asset_Data_New['Unit #']
        if Generation_Asset_Data_New.loc[Generation_Asset_Data_New['Unit #'] == i, 'Technology'].values[0] == 'PV'
    ),
    "PVGenMax"
)

#generation constraint for assets with technology=Wind	
m.addConstrs(
    (
        P_NewCap[i, y, s, h] <= Inv_NewCap[i] * VRE_Generation_profile.loc[
            (VRE_Generation_profile['Hour'] == h+1) & (VRE_Generation_profile['Season'] == s+1), 'WindZ1'
        ].values[0]
        for y in range(Y)
        for s in range(S)
        for h in range(H)
        for i in Generation_Asset_Data_New['Unit #']
        if Generation_Asset_Data_New.loc[Generation_Asset_Data_New['Unit #'] == i, 'Technology'].values[0] == 'Wind'
    ),
    "WindGenMax"
)

#Solve the model
m.optimize()

# Check if the model found an optimal solution
if m.status == GRB.OPTIMAL:
    # Print the optimal solution
    for v in m.getVars():
        print('%s %g' % (v.varName, v.x))
    print('Obj:', m.objVal)
else:
    print("No optimal solution found. Status code:", m.status)

# Save the optimal solution to a df
# First df contains the investment decisions
df1 = pd.DataFrame(columns=['Unit #', 'Investment'])
df1['Unit #'] = Generation_Asset_Data_New['Unit #']
df1['Investment'] = [Inv_NewCap[i].x for i in Generation_Asset_Data_New['Unit #']]

# Second df contains the generation decisions
df2 = pd.DataFrame(columns=['Unit #', 'Year', 'Season', 'Hour', 'Generation'])
df2['Unit #'] = [i for i in Generation_Asset_Data_New['Unit #'] for y in range(Y) for s in range(S) for h in range(H)]
df2['Year'] = [y for i in Generation_Asset_Data_New['Unit #'] for y in range(Y) for s in range(S) for h in range(H)]
df2['Season'] = [s for i in Generation_Asset_Data_New['Unit #'] for y in range(Y) for s in range(S) for h in range(H)]
df2['Hour'] = [h for i in Generation_Asset_Data_New['Unit #'] for y in range(Y) for s in range(S) for h in range(H)]
df2['Generation'] = [P_NewCap[i, y, s, h].x for i in Generation_Asset_Data_New['Unit #'] for y in range(Y) for s in range(S) for h in range(H)]

print(df1)
print(df2)






Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads



GurobiError: Model too large for size-limited license; visit https://gurobi.com/unrestricted for more information

In [56]:
m.write("model.lp")